# Converting to Fantasy Points

In [ ]:
import pandas as pd

In [ ]:
player_proj = pd.read_csv('../nba-stats-csv/player_proj_df.csv')
player_info = pd.read_csv('../nba-stats-csv/player_info_df.csv')

In [ ]:
proj_columns = [
    'proj_pts',
    'proj_min',
    'proj_fgm',
    'proj_fga',
    'proj_fg3m',
    'proj_fg3a',
    'proj_ftm',
    'proj_fta',
    'proj_oreb',
    'proj_dreb',
    'proj_ast',
    'proj_stl',
    'proj_tov',
    'proj_blk'                     
]

In [ ]:
df_proj_stats = player_proj.loc[:, proj_columns]

In [ ]:
df_proj_stats = pd.concat([player_info, df_proj_stats], axis=1)

In [ ]:
proj_dictionary = {  'proj_season_id' : 'season_id',
                'proj_pts' : 'pts',
                'proj_min' : 'min',
                'proj_fgm' : 'fgm',
                'proj_fga' : 'fga',
                'proj_fg3m' : 'fg3m',
                'proj_fg3a' : 'fg3a',
                'proj_ftm' : 'ftm',
                'proj_fta' : 'fta',
                'proj_oreb' : 'oreb',
                'proj_dreb' : 'dreb',
                'proj_ast' : 'ast',
                'proj_stl' : 'stl',
                'proj_tov' : 'tov',
                'proj_blk' : 'blk'
            }

In [ ]:
df_proj_stats.rename(columns = proj_dictionary, inplace=True)

In [ ]:
df_proj_stats.sample(5)

In [ ]:
def fantasy_pts_converter(row):
    pts = row['pts']
    fgm = row['fgm']
    fga = row['fga']
    ftm = row['ftm']
    fta = row['fta']
    oreb = row['oreb']
    dreb = row['dreb']
    ast = row['ast']
    stl = row['stl']
    tov = row['tov']
    blk = row['blk']
    fantasy_pts = pts + fgm - fga + ftm - fta + oreb + dreb + ast + stl - tov + blk
    return fantasy_pts

In [ ]:
fantasy_value = df_proj_stats.apply(fantasy_pts_converter, axis = 1)

In [ ]:
df_proj_stats['fantasy_pts'] = fantasy_value

In [ ]:
df_proj_stats.sort_values('fantasy_pts', ascending=False)

In [ ]:
df_for_export = round(df_proj_stats.sort_values('fantasy_pts', ascending=False),2)

In [ ]:
df_for_export.to_csv('../nba-stats-csv/projected_fantasy_pts_final_2018_19.csv', index=False)

In [ ]:
df_for_export.to_csv('../nba-stats-csv/projected_fantasy_pts_final_2018_19_flan02.csv', index=False)

# Value Based Drafting

In [ ]:
# The Principle:
# The value of a player is determined not by the number of points he scored, but by how much he outscores his peers at his particular position.

| Your Team | Opponent | Points Advantage | Cumulative Total |
| --------- | -------- | ---------------- | ---------------- |
|PG scores 23 pts| PG scores 20 pts| +3 for you | +3 |
|SG scores 7 pts| SG scores 13 pts| -6 for you | -3 |
|SF scores 20 pts| SF scores 5 pts| +15 for you | +12 |
|PF scores 18 pts| PF scores 19 pts| -1 for you | +11 |
|C scores 30 pts| C scores 34 pts| -4 for you | +7 |

##### Total Pts = 98 (Your Team); Total Pts = 91 (Opponent)

# Getting Our Baseline Numbers

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
draft_df = pd.read_csv('../nba-stats-csv/Fantasy Projections - 2018_19 - 2018_espn_draft.csv', header=0)

In [ ]:
draft_df.head(5)

In [ ]:
df_top_80 = draft_df[draft_df['pick'] <= 80]

In [ ]:
df_top_80.tail()

In [ ]:
df_top_80.groupby('position').count() # How many players of each position do we have in this draft?

In [ ]:
position_dictionary = {
    'PG': 21,
    'SG': 13,
    'SF': 12,
    'PF': 19,
    'C': 15
}

In [ ]:
df1 = pd.read_csv('../nba-stats-csv/Fantasy Projections - 2018_19 - 2018_espn_player_positions.csv', header=0)
df2 = pd.read_csv('../nba-stats-csv/projected_fantasy_pts_final_2018_19.csv', header=0)

In [ ]:
df1.sample()

In [ ]:
df2.sample()

In [ ]:
df3 = df1.merge(df2, how='left', on='player_id')

In [ ]:
df_final = df3[['player_name_x', 'player_id', 'season_id_x', 'espn_position', 'fantasy_pts']]

In [ ]:
df_final = df_final.rename(columns={
    'player_name_x': 'player_name',
    'season_id_x': 'season_id'
})

In [ ]:
df_final.to_csv('../nba-stats-csv/projected_fantasy_pts_with_positions_2018_19_formatted.csv', index=False)

In [ ]:
positions = [
    'PG',
    'SG',
    'SF',
    'PF',
    'C'
]

In [ ]:
def calc_baseline(pos, df):
    df_filter = df[df['espn_position'] == pos]
    x = position_dictionary.get(pos)
    replacement_player = df_filter.iloc[x]
    value_to_substract = replacement_player['fantasy_pts']
    return value_to_substract

In [ ]:
baseline_points = {}
for pos in positions:
    y = calc_baseline(pos, df_final)
    baseline_points[pos] = y

In [ ]:
baseline_points

# Draft Preparation

In [ ]:
# We can work with model importing our .csv file to spreadsheet or excel and manipulate it